<a href="https://colab.research.google.com/github/dhkimxx/bigData-analysis-examples/blob/main/analysisExample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
x_train = pd.read_csv('x_train.csv', encoding = 'CP949')
y_train = pd.read_csv('y_train.csv', encoding = 'CP949')
x_test = pd.read_csv('x_test.csv', encoding = 'CP949')

x_test_cust_id = x_test['cust_id']
x_train = x_train.drop(columns = ['cust_id'])
y_train = y_train.drop(columns = ['cust_id'])
x_test = x_test.drop(columns = ['cust_id'])

x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])

condition = x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
x_train = x_train.drop(columns = ['환불금액'])
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0
x_test = x_test.drop(columns = ['환불금액'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

#print(x_train[['총구매액','최대구매액','환불금액_new']].corr()) #상관관계가 0.6 이상이면 강한 상관관계 -> 다중공선성
x_train = x_train.drop(columns = ['최대구매액'])
x_test = x_test.drop(columns = ['최대구매액'])

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth = 10, criterion = 'entropy')
model.fit(x_train, y_train)
y_test_predicted = model.predict(x_test)
y_test_proba = model.predict_proba(x_test)
result = pd.DataFrame(y_test_proba)[1]

y_train_predicted = model.predict(x_train)
from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y_train, y_train_predicted))

final = pd.concat([x_test_cust_id, result], axis = 1).rename(columns = {0: 'cust_id', 1: 'gender'}) #열방향으로 붙이기
final.to_csv('12345.csv', index = False)

final = pd.read_csv('12345.csv')
print(final)

      cust_id    gender
0        3500  0.800000
1        3501  0.000000
2        3502  1.000000
3        3503  0.451163
4        3504  0.000000
...       ...       ...
2477     5977  0.462312
2478     5978  0.000000
2479     5979  0.338028
2480     5980  0.000000
2481     5981  1.000000

[2482 rows x 2 columns]
